In [ ]:
import glob, os
import ENDFtk
from ENDFtk import tree
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib widget

tape = ENDFtk.tree.Tape.from_file( '/home/pierre/Projects/LIB/JEFF33/26-Fe-56g.jeff33' )
mf4mt2 = tape.MAT(tape.material_numbers[0]).MF(4).MT(2).parse()
angulard = tape.materials.front().section( 4, 2 ).parse()
angularu = tape.materials.front().section( 34, 2 ).parse()
dist = angulard.distributions.legendre.incident_energies

def plot_legendre_coeffs(angulard, orders):
    """
    Plot Legendre coefficients for given order(s) as a function of incident energy.

    Parameters:
    angulard: parsed angular distribution object
    orders: int or list of ints, Legendre order(s) to plot
    """
    if isinstance(orders, int):
        orders = [orders]

    energies = [dist.incident_energy for dist in angulard.angular_distributions.to_list()]
    max_order = max(len(dist.coefficients[:]) for dist in angulard.distributions.legendre.angular_distributions.to_list())
    coeff_array = np.zeros((len(energies), max_order))

    for i, dist in enumerate(angulard.distributions.legendre.angular_distributions.to_list()):
        coeffs = dist.coefficients
        coeff_array[i, :len(coeffs)] = coeffs

    for l in orders:
        if l < max_order:
            plt.plot(energies, coeff_array[:, l], label=r'$a_{l=%d}$' % l)
        else:
            print(f"Order {l} exceeds available maximum order {max_order-1}")

    plt.xscale('log')
    plt.xlim(1000,5e7)
    plt.xlabel('Energy [eV]')
    plt.ylabel('Legendre Coefficient aₗ')
    plt.title(r'$^{26}Al$ Elastic Angular Distributions')
    plt.legend()
    plt.show()
    
# plot_legendre_coeffs(angulard, [0, 1, 2, 3, 4, 5, 6])

In [ ]:
def plot_superposed_legendre_enhanced(file_list, legendre_order, emin, emax, 
                                      reference_file=None, reference_pattern='*_random0.endf',
                                      nominal_tape_path=None, max_samples=None, 
                                      figsize=(12, 8), alpha_samples=0.6, 
                                      linewidth_ref=3, linewidth_samples=1.5,
                                      save_plot=False, save_path=None):
    """
    Enhanced version with better reference file handling and save option.
    
    Parameters
    ----------
    file_list : list of str
        List of file paths to process
    legendre_order : int
        Legendre order to plot (0-based indexing)
    emin, emax : float
        Energy range for x-axis limits (eV)
    reference_file : str, optional
        Explicit path to reference file (overrides pattern search)
    reference_pattern : str
        Pattern to identify the reference file in file_list
    nominal_tape_path : str, optional
        Path to original nominal tape (used if no reference found in file_list)
    max_samples : int, optional
        Maximum number of sample files to plot
    save_plot : bool
        Whether to save the plot to file
    save_path : str, optional
        Path to save the plot (if None, auto-generate name)
    
    Returns
    -------
    fig, ax : matplotlib figure and axis objects
    plot_info : dict with plotting statistics
    """
    
    # Determine reference file
    sample_files = []
    ref_file = None
    ref_source = "unknown"
    
    if reference_file is not None:
        # Explicit reference provided
        ref_file = reference_file
        sample_files = [f for f in file_list if f != reference_file]
        ref_source = "explicit"
    else:
        # Search for reference in file_list
        for filepath in file_list:
            basename = os.path.basename(filepath)
            if reference_pattern.replace('*', '') in basename:
                ref_file = filepath
                ref_source = "pattern_match"
                break
        
        # Set sample files (excluding reference)
        sample_files = [f for f in file_list if f != ref_file]
        
        # If no reference found in list, try nominal tape
        if ref_file is None and nominal_tape_path is not None:
            ref_file = nominal_tape_path
            sample_files = file_list[:]  # All files become samples
            ref_source = "nominal_tape"
        elif ref_file is None:
            # Use first file as reference
            ref_file = file_list[0]
            sample_files = file_list[1:]
            ref_source = "first_file"
    
    # Limit samples if requested
    if max_samples is not None:
        sample_files = sample_files[:max_samples]
    
    print(f"Reference file: {os.path.basename(ref_file)} (source: {ref_source})")
    print(f"Sample files: {len(sample_files)} files")
    
    # Create figure
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    
    # Extract data function (same as before)
    def extract_legendre_data(filepath):
        try:
            tape = tree.Tape.from_file(filepath)
            mf4mt2 = tape.MAT(tape.material_numbers[0]).MF(4).MT(2).parse()
            
            if mf4mt2.LTT == 1:
                legendre_dist = mf4mt2.distributions
            elif mf4mt2.LTT == 3:
                legendre_dist = mf4mt2.distributions.legendre
            else:
                raise ValueError(f"Unsupported LTT={mf4mt2.LTT}")
            
            angular_distributions = legendre_dist.angular_distributions.to_list()
            energies, coefficients = [], []
            
            for dist in angular_distributions:
                energy = dist.incident_energy
                coeffs = dist.coefficients[:]
                energies.append(energy)
                
                if legendre_order < len(coeffs):
                    coefficients.append(coeffs[legendre_order])
                else:
                    coefficients.append(0.0)
            
            return np.array(energies), np.array(coefficients)
            
        except Exception as e:
            print(f"Error processing {os.path.basename(filepath)}: {e}")
            return None, None
    
    # Plot reference
    ref_energies, ref_coeffs = extract_legendre_data(ref_file)
    ref_label = f'Reference ({os.path.basename(ref_file)})'
    if ref_source == "nominal_tape":
        ref_label = f'Nominal ({os.path.basename(ref_file)})'
    
    if ref_energies is not None:
        ax.plot(ref_energies, ref_coeffs, 
                color='black', linewidth=linewidth_ref, linestyle='-',
                label=ref_label, zorder=10)
    
    # Plot samples
    colors = plt.cm.tab10(np.linspace(0, 1, min(10, len(sample_files))))
    valid_samples = 0
    
    for i, filepath in enumerate(sample_files):
        energies, coeffs = extract_legendre_data(filepath)
        if energies is not None:
            color = colors[i % len(colors)] if len(colors) > 0 else f'C{i}'
            sample_label = f'Sample {i+1}' if valid_samples < 8 else ''
            ax.plot(energies, coeffs,
                    color=color, linewidth=linewidth_samples, linestyle='--',
                    label=sample_label, alpha=alpha_samples, zorder=5)
            valid_samples += 1
    
    # Set up plot
    ax.set_xlabel('Energy (eV)', fontsize=12)
    ax.set_ylabel(f'L={legendre_order} Legendre Coefficient', fontsize=12)
    title = f'L={legendre_order} Coefficient vs Energy - Superposed Samples\\n'
    title += f'{ref_source.replace("_", " ").title()} vs {valid_samples} samples'
    ax.set_title(title, fontsize=14)
    ax.set_xscale('log')
    ax.set_xlim(emin, emax)
    ax.grid(True, alpha=0.3)
    
    # Legend handling
    if valid_samples <= 8:
        ax.legend(fontsize=9, loc='best')
    else:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend([handles[0]], [labels[0]], fontsize=10, loc='best')
        ax.text(0.02, 0.98, f'+ {valid_samples} sample files', 
                transform=ax.transAxes, fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    ax.tick_params(labelsize=10)
    plt.tight_layout()
    
    # Save plot if requested
    if save_plot:
        if save_path is None:
            save_path = f'legendre_L{legendre_order}_superposed_E{emin:.0e}-{emax:.0e}.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved to: {save_path}")
    
    # Prepare return info
    plot_info = {
        'reference_file': ref_file,
        'reference_source': ref_source,
        'num_samples': valid_samples,
        'legendre_order': legendre_order,
        'energy_range': (emin, emax),
        'save_path': save_path if save_plot else None
    }
    
    print(f"Successfully plotted {valid_samples} samples + 1 reference")
    return fig, ax, plot_info

def plot_random_samples_comparison(legendre_order, emin=1e-5, emax=2e7, sample_pattern='sampled_tape_random*.endf', max_samples=20, save=False):
    
    # Find all sample files
    sample_files = sorted(glob.glob(sample_pattern))
    
    if len(sample_files) == 0:
        print(f"No files found matching pattern: {sample_pattern}")
        return None, None, None
    
    print(f"Found {len(sample_files)} files matching '{sample_pattern}'")
    
    # Use the enhanced function
    fig, ax, info = plot_superposed_legendre_enhanced(
        file_list=sample_files,
        legendre_order=legendre_order,
        emin=emin,
        emax=emax,
        nominal_tape_path=tape_adress if 'tape_adress' in globals() else None,
        max_samples=max_samples,
        save_plot=save
    )
    
    return fig, ax, info

%matplotlib widget

fig, ax, info = plot_random_samples_comparison(
    legendre_order=1,  # L=0 coefficient
    emin=1e3,
    emax=2.5e7,
    # max_samples=12
)

plt.show()